# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'sj-200418-1628'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [3]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

Reminder: the tuning job has not been completed.
111 training jobs have completed


In [4]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2020, 4, 19, 3, 34, 11, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'test:RMSE',
                                                 'Value': 0.7468509078025818},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2020, 4, 19, 4, 6, 3, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:eu-central-1:964501460451:training-job/sj-200418-1628-145-8b039a0e',
 'TrainingJobName': 'sj-200418-1628-145-8b039a0e',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2020, 4, 19, 3, 35, 56, tzinfo=tzlocal()),
 'TunedHyperParameters': {'context_length': '64',
                          'dropout_rate': '0.010630880176942881',
                          'learning_rate': '0.00891170222611896',
                          'mini_batch_size': '41',
                          'num_cells': '115'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [5]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 111
{'lowest': 0.7468509078025818, 'highest': 135.62213134765625}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,context_length,dropout_rate,learning_rate,mini_batch_size,num_cells,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
59,64.0,0.010631,0.008912,41.0,115.0,sj-200418-1628-145-8b039a0e,Completed,0.746851,2020-04-19 03:35:56+00:00,2020-04-19 04:06:03+00:00,1807.0
123,248.0,0.031769,0.005520,32.0,123.0,sj-200418-1628-081-ebbc11dc,Completed,0.985254,2020-04-18 22:42:11+00:00,2020-04-18 23:30:43+00:00,2912.0
45,256.0,0.017460,0.004559,37.0,83.0,sj-200418-1628-159-499c5d41,Completed,1.005499,2020-04-19 05:01:29+00:00,2020-04-19 05:41:31+00:00,2402.0
33,253.0,0.010571,0.006670,61.0,112.0,sj-200418-1628-171-de49060b,Completed,1.397323,2020-04-19 05:45:14+00:00,2020-04-19 06:30:30+00:00,2716.0
42,124.0,0.013587,0.006638,41.0,114.0,sj-200418-1628-162-1ae5e441,Completed,1.413471,2020-04-19 05:07:53+00:00,2020-04-19 05:50:00+00:00,2527.0
...,...,...,...,...,...,...,...,...,...,...,...
160,256.0,0.070874,0.033238,47.0,107.0,sj-200418-1628-044-48cb9d05,Completed,45.141689,2020-04-18 19:51:01+00:00,2020-04-18 20:10:44+00:00,1183.0
201,41.0,0.017781,0.022463,98.0,85.0,sj-200418-1628-003-e3e556b4,Completed,49.755692,2020-04-18 16:31:35+00:00,2020-04-18 17:28:51+00:00,3436.0
200,187.0,0.019713,0.072496,65.0,70.0,sj-200418-1628-004-e3724b08,Completed,50.146233,2020-04-18 16:30:48+00:00,2020-04-18 16:52:07+00:00,1279.0
74,16.0,0.087045,0.015614,36.0,43.0,sj-200418-1628-130-16d6582b,Completed,55.330357,2020-04-19 02:55:44+00:00,2020-04-19 03:08:48+00:00,784.0


In [6]:
df  = df.sort_values('FinalObjectiveValue')[:-2]

## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [7]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [8]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))